In [29]:
def count_valid_solutions(n, min_interval, max_interval, last_start, last_end):
    dp = [0] * n
    # 初始化第一个1的位置在min_interval+1到max_interval+1之间
    for k in range(min_interval + 1, max_interval + 2):
        dp[k] = 1
    # 填充动态规划数组
    for k in range(min_interval + 1 + 1, n):
        for j in range(max(k - (max_interval + 1), min_interval + 1), k - (min_interval + 1) + 1):
            if 0 <= j < n:
                dp[k] += dp[j]
    # 计算最后1在last_start到last_end之间的解的数量
    total = sum(dp[last_start:last_end + 1])
    return total

# 参数设置
n = 96
min_interval = 11  # 对应11个0,距离为12
max_interval = 15  # 对应15个0,距离为16
last_start = 84
last_end = 95

# 计算解的数量
total_solutions = count_valid_solutions(n, min_interval, max_interval, last_start, last_end)
print(f"Total number of valid solutions: {total_solutions}")

Total number of valid solutions: 28667


In [30]:
import gurobipy as gp
from gurobipy import GRB

# 参数设置 interval 代表1之间间隔的0的数量
min_interval = 11    # 对应3h的情况
max_interval = 15    # 对应4h的情况
n = 96

# 创建模型
model = gp.Model('optimization_model')

# 创建变量
x = model.addVars(n, vtype=GRB.BINARY, name='x')  # x[0] 到 x[95] 分别代表96个位置是否为1

# 约束条件

# 条件1: 前 min_interval 个位置必须是0
for i in range(min_interval):
    model.addConstr(x[i] == 0)

# 条件2: 在 min_interval 到 max_interval 之间必须有一个1
model.addConstr(gp.quicksum(x[i] for i in range(min_interval, max_interval)) == 1)

# 条件3a: 两个1之间的最小距离至少是 min_interval
for i in range(n):
    for j in range(i + 1, i + min_interval):
        if j > n-1:
            break
        model.addConstr(x[i] + x[j] <= 1)

# 条件3b: 两个1之间的最大距离不超过 max_interval
for i in range(n - max_interval):
    model.addConstr(gp.quicksum(x[j] for j in range(i, i + max_interval)) >= 1)

# 目标函数: 设为0,因为我们不关心目标值,只寻找可行解
model.setObjective(0, GRB.MAXIMIZE)

# 设置求解参数
model.Params.PoolSearchMode = 2  # 寻找所有解决方案
model.Params.PoolSolutions = 10000  # 设置一个足够大的上限
model.Params.PoolGap = 0.0  # 找到所有解决方案,不考虑目标值差距

# 求解模型
model.optimize()

# 输出结果
if model.status == GRB.OPTIMAL:
    print('Number of solutions found: ', model.solCount)
    # for sol in range(model.solCount):
    #     model.setParam(GRB.Param.SolutionNumber, sol)
    #     solution = [x[i].Xn for i in range(n)]
    #     print('Solution ', sol + 1, ':', end=' ')
    #     for i in range(n):
    #         if solution[i] == 1:
    #             print(i + 1, end=' ')
    #     print()
else:
    print('No feasible solution found.')
    

Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 10000
Set parameter PoolGap to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1155G7 @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 998 rows, 96 columns and 3040 nonzeros
Model fingerprint: 0xb0c22f9b
Variable types: 0 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 876 rows and 22 columns
Presolve time: 0.01s
Presolved: 122 rows, 74 columns, 1472 nonzeros
Variable types: 0 continuous, 74 integer (74 binary)
Found heuristic solution: objective -0.0000000

Root relaxation: objective -0.000000e+00, 26 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Curr

In [32]:
# 输出结果
print('12: 2:45~3:00  16: 3:45~4:00 20: 4:45~5:00 24: 5:45~6:00 28: 6:45~7:00 32: 7:45~8:00 36: 8:45~9:00 40: 9:45~10:00 44: 10:45~11:00 48: 11:45~12:00 52: 12:45~13:00 56: 13:45~14:00 60: 14:45~15:00 64: 15:45~16:00 68: 16:45~17:00 72: 17:45~18:00 76: 18:45~19:00 80: 19:45~20:00 84: 20:45~21:00 88: 21:45~22:00 92: 22:45~23:00 96: 23:45~24:00')
if model.status == GRB.OPTIMAL:
    print('Number of solutions found: ', model.solCount)
    for sol in range(model.solCount):
        model.setParam(GRB.Param.SolutionNumber, sol)
        solution = [x[i].Xn for i in range(n)]
        print('Solution ', sol + 1, ':', end=' ')
        for i in range(n):
            if solution[i] == 1:
                print(i + 1, end=' ')
        solution = [int(x[i].Xn) for i in range(n)]
        print()
        print(solution)
        print('=========================')
else:
    print('No feasible solution found.')

12: 2:45~3:00  16: 3:45~4:00 20: 4:45~5:00 24: 5:45~6:00 28: 6:45~7:00 32: 7:45~8:00 36: 8:45~9:00 40: 9:45~10:00 44: 10:45~11:00 48: 11:45~12:00 52: 12:45~13:00 56: 13:45~14:00 60: 14:45~15:00 64: 15:45~16:00 68: 16:45~17:00 72: 17:45~18:00 76: 18:45~19:00 80: 19:45~20:00 84: 20:45~21:00 88: 21:45~22:00 92: 22:45~23:00 96: 23:45~24:00
Number of solutions found:  10000
Solution  1 : 15 30 45 59 70 81 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Solution  2 : 13 28 41 52 63 74 85 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,

## 优化问题描述

### 目标函数

不设置具体目标函数,仅寻找满足约束条件的可行解.

$$ \text{最大化} \quad 0 $$

### 决策变量

定义二元变量 $x_i \in \{0, 1\}$,表示在第 $i$ 个位置是否放置1,其中 $i = 0, 1, \dots, 95$.

### 约束条件

1. **前 min_interval 个位置必须是0:**

   $$ x_i = 0, \quad \text{对于} \quad i = 0, 1, \dots, \text{min\_interval}-1 $$

2. **在 min_interval 到 max_interval 之间必须有一个1:**

   $$ \sum_{i=\text{min\_interval}}^{\text{max\_interval}-1} x_i = 1 $$

3. **两个1之间的最小距离至少是 min_interval:**

   $$ x_i + x_j \leq 1, \quad \text{对于所有} \quad j \in \{i+1, i+2, \dots, i+\text{min\_interval}-1\}, \quad \text{且} \quad j < n $$

4. **两个1之间的最大距离不超过 max_interval:**

   $$ \sum_{j=i}^{i+\text{max\_interval}-1} x_j \geq 1, \quad \text{对于所有} \quad i = 0, 1, \dots, n-\text{max\_interval} $$

5. **最右侧的1的位置要在84-96之间:**

   $$ \sum_{i=84}^{95} x_i \geq 1 $$

### 参数设置

- $n = 96$
- $\text{min\_interval} = 11$
- $\text{max\_interval} = 15$

### 求解设置

- 求解所有可行解
- 设置解的上限为10000
- 不考虑目标值差距,找到所有解

### 模型求解

使用Gurobi求解器求解模型,并输出所有找到的解.